In [1]:
from itertools import chain


import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

from utils import *

In [2]:
train_file = "../data/train.csv"
test_file = "../data/test.csv"
sample_submission = "../data/sample_submission.csv"
submission_filename = "test_submission.csv"

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.8,
    'silent': 0,
    'subsample': .85,
    'eta': 0.0275,
    'objective': 'binary:logitraw',
    'num_parallel_tree': 7,
    'max_depth': 5,
    'nthread': 22,
    'eval_metric': 'auc',
}


In [3]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn import preprocessing

train_=pd.read_csv('../data/train.csv',delimiter=',')
test_=pd.read_csv('../data/test.csv',delimiter=',')
pd.options.mode.chained_assignment = None
Id_ts=test_['ID']
test_.drop('ID',axis=1,inplace=True)


clean(train_,test_)
Target=train_['Target']
train_.drop('OutcomeSubtype',axis=1,inplace=True)
train_.drop('Aggressive',axis=1,inplace=True)
train_.drop('At Vet',axis=1,inplace=True)
train_.drop('Barn',axis=1,inplace=True)
train_.drop('Behavior',axis=1,inplace=True)
train_.drop('Court/Investigation',axis=1,inplace=True)
train_.drop('Enroute',axis=1,inplace=True)
train_.drop('Foster',axis=1,inplace=True)
train_.drop('In Foster',axis=1,inplace=True)
train_.drop('In Kennel',axis=1,inplace=True)
train_.drop('In Surgery',axis=1,inplace=True)
train_.drop('Medical',axis=1,inplace=True)
train_.drop('Offsite',axis=1,inplace=True)
train_.drop('Partner',axis=1,inplace=True)
train_.drop('Rabies Risk',axis=1,inplace=True)
train_.drop('SCRP',axis=1,inplace=True)
train_.drop('Suffering',axis=1,inplace=True)
train_.drop('OutcomeType',axis=1,inplace=True)
train_.drop('Target',axis=1,inplace=True)

train_2 = clean_v2(train_)
train_2 = pd.get_dummies(train_2, columns=['Groups','App','Family','Hairless','Hypoa','Kids','Smart','Type2','DG',
                                           'Type4','dayofweek','Holidays','Season','am_pm','Unicolor','month','day',
                                           'Top_Mix','AgeuponOutcome_cat','sprayed','Type_sex','SexuponOutcome_cat',
                                           'AnimalType_cat'])

test_2=clean_v2(test_)
test_2 = pd.get_dummies(test_2, columns=['Groups','App','Family','Hairless','Hypoa','Kids','Smart','Type2','DG',
                                         'Type4','dayofweek','Holidays','Season','am_pm','Unicolor','month','day',
                                         'Top_Mix','AgeuponOutcome_cat','sprayed','Type_sex','SexuponOutcome_cat',
                                         'AnimalType_cat'])



In [10]:

train_2['version']='Train'
test_2['version']='Test'

result = train_2.append(test_2, ignore_index=True)

binarize = result

binarize_train=binarize[binarize['version']=='Train']
binarize_test=binarize[binarize['version']=='Test']
print(binarize.shape)

train_select=pd.DataFrame(binarize_train.select_dtypes(include=['float64','int64','int32','float32']))
test_select=pd.DataFrame(binarize_test.select_dtypes(include=['float64','int64','int32','float32']))

print(train_select.shape)

train_select=train_select.fillna(-9999)
test_select=test_select.fillna(-9999)

x_train, x_test, y_train, y_test = train_test_split(train_select,Target,test_size=0.30, 
                                                    random_state=30,stratify=Target)

dtrain = xgb.DMatrix(x_train,y_train,missing = -9999)
dtest = xgb.DMatrix(x_test,y_test,missing = -9999)

param = {'max_depth':7, 'eta':0.1, 'silent':1, 'objective':'multi:softprob','num_class':5,
        'eval_metric':'mlogloss','subsample':0.75,'colsample_bytree':0.85}
#watchlist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 125

bst1 = xgb.train(param, dtrain, num_round)


(38185, 172)
(26729, 154)


In [12]:
ypred_submit = bst1.predict(dtest)
ypred_submit = pd.DataFrame(ypred_submit)

submission = pd.DataFrame()
submission["id"] = Id_ts.values
submission["Adoption"]= ypred_submit[2]
submission["Died"]= ypred_submit[4]
submission["Euthanasia"]= ypred_submit[1]
submission["Return_to_owner"]= ypred_submit[0]
submission["Transfer"]= ypred_submit[3]

submission.to_csv("sub.csv",index=False)